In [53]:
from __future__ import unicode_literals
import classifiers
import re
import nltk
from Politweet import get_tweets, get_transcript
import ratings
from sentiment import polarity_train, classify, prob_classify, plus_df, minus_df
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

In [54]:
tweets = get_tweets("./datasets/tweets.tsv")
transcript = get_transcript("./datasets/transcript.csv")
todo = set(tweets.index)


In [55]:
tweets.to_pickle('./datasets/tweets.pickle')

In [56]:
polarity = polarity_train(tweets)
polarity_minus = set(i for i,_ in classify(minus_df(tweets), polarity))
polarity_plus = set(i for i,_ in classify(plus_df(tweets), polarity))
polarities = tweets.reindex(polarity_minus | polarity_plus).sort()
print len(polarities), "- polarity"
todo -= set(polarities.index)

482 - polarity


In [57]:
others = ratings.all(tweets, ratings.OTHER)
print len(others), "- marked as OTHERS"
todo -= set(others.index)

151 - marked as OTHERS


In [58]:
print len(todo), "- untagged"

2605 - untagged


In [59]:
left = tweets.reindex(todo).sort()
left[:1]

,pub.date.GMT,content,author.name,author.nickname,rating.1,rating.2,rating.3,rating.4,rating.5,rating.6,rating.7,rating.8,clean,tokens
tweet.id,,,,,,,,,,,,,,
936466790,9/27/08 1:00,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,B_E_X,Becky L,4,2,4,NaN,NaN,NaN,NaN,NaN,"[{u'lemma': u'jim', u'token': u'jim', u'pos': u'NN'}, {u'lemma': u'lehrer', u'token': u'lehrer', u'pos': u'NN'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]","[jim, lehrer, direct, debate, audience, second, tweetdebate]"


In [60]:
left[["content","tokens", "clean"]]

,content,tokens,clean
tweet.id,,,
936466790,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,"[jim, lehrer, direct, debate, audience, second, tweetdebate]","[{u'lemma': u'jim', u'token': u'jim', u'pos': u'NN'}, {u'lemma': u'lehrer', u'token': u'lehrer', u'pos': u'NN'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936466992,Here we go. #tweetdebate,[tweetdebate],"[{u'lemma': u'go', u'token': u'go', u'pos': u'VBP'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936467521,#tweetdebate letting mom/dad follow debate on CNN while tweet scoring. I will sleep on the sofa with them & wait for the screams GO Obama,"[tweetdebate, let, momdad, follow, debate, cnn, tweet, scoring, sleep, sofa, wait, scream, obama]","[{u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}, {u'lemma': u'let', u'token': u'letting', u'pos': u'VBG'}, {u'lemma': u'momdad', u'token': u'momdad', u'pos': u'JJ'}, {u'lemma': u'follow', u'token': u'follow', u'pos': u'NN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'cnn', u'token': u'cnn', u'pos': u'NN'}, {u'lemma': u'tweet', u'token': u'tweet', u'pos': u'NN'}, {u'lemma': u'scoring', u'token': u'scoring', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'sleep', u'token': u'sleep', u'pos': u'VB'}, {u'lemma': u'sofa', u'token': u'sofa', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u'CC'}, {u'lemma': u'wait', u'token': u'wait', u'pos': u'VB'}, {u'lemma': u'scream', u'token': u'screams', u'pos': u'NNS'}, {u'lemma': u'go', u'token': u'go', u'pos': u'VBP'}, {u'lemma': u'obama', u'token': u'obama', u'pos': u'NN'}]"
936467712,what is an appropriately bi-partisan food to bring to a debate party? #tweetdebate,"[appropriately, bipartisan, food, bring, debate, party, tweetdebate]","[{u'lemma': u'appropriately', u'token': u'appropriately', u'pos': u'RB'}, {u'lemma': u'bipartisan', u'token': u'bipartisan', u'pos': u'JJ'}, {u'lemma': u'food', u'token': u'food', u'pos': u'NN'}, {u'lemma': u'bring', u'token': u'bring', u'pos': u'VB'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'party', u'token': u'party', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936468736,@current I've got the live stream going #current....awesome!,"[current, get, live, stream, current, awesome]","[{u'lemma': u'', u'token': u'', u'pos': u'IN'}, {u'lemma': u'current', u'token': u'current', u'pos': u'JJ'}, {u'lemma': u've', u'token': u've', u'pos': u'VBP'}, {u'lemma': u'get', u'token': u'got', u'pos': u'VBN'}, {u'lemma': u'live', u'token': u'live', u'pos': u'JJ'}, {u'lemma': u'stream', u'token': u'stream', u'pos': u'NN'}, {u'lemma': u'go', u'token': u'going', u'pos': u'VBG'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'current', u'token': u'current', u'pos': u'JJ'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'awesome', u'token': u'awesome', u'pos': u'JJ'}, {u'lemma': u'', u'token': u'', u'pos': u''}]"
936468824,watching the debate on current TV in SF. let's go! #tweetdebate,"[watch, debate, current, let, tweetdebate]","[{u'lemma': u'watch', u'token': u'watching', u'pos': u'VBG'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'current', u'token': u'curre

In [61]:
for i, tweet in tweets[:3].iterrows():
    print [token for token in tweet["tokens"]]
    
tokens = [" ".join(tweet["tokens"]) for i,tweet in tweets.iterrows()]
tokens[:1]

[u'watch', u'tweetdebate', u'drink', u'wait', u'start', u'cringing', u'mccain', u'blunder']
[u'ahg3', u'michdot', u'yeah', u'slime', u'actually', u'second', u'choice', u'say', u'first', u'one', u'okay', u'roll']
[u'prepare', u'heart', u'attack', u'tweetdebate']


[u'watch tweetdebate drink wait start cringing mccain blunder']

In [107]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
count = Counter([token for i, tweet in tweets.iterrows() for token in tweet["tokens"]])

vectorizer = TfidfVectorizer(lowercase=False)
tfidf_model = vectorizer.fit_transform(tokens)
feature_names = vectorizer.get_feature_names()
df = pd.DataFrame([
        (feature_names[col], tfidf_model[0, col])
        for col in tfidf_model.nonzero()[1]])
df.sort(1, ascending=False)

,0,1
2,cringing,0.528286
0,blunder,0.484641
9227,blunder,0.484641
4504,blunder,0.484641
104,start,0.351767
19441,drink,0.351767
8400,drink,0.351767
20100,drink,0.351767
20081,start,0.351767
6630,drink,0.351767


In [127]:
for col in tfidf_model.nonzero()[1]:
    print tfidf_model[0, col]
    print (feature_names[col], tfidf_model[0, col])

0.484640987077
(u'blunder', 0.48464098707655739)
0.127842163653
(u'mccain', 0.12784216365339229)
0.528286499865
(u'cringing', 0.52828649986536869)
0.351766844606
(u'start', 0.35176684460591617)
0.335099904907
(u'wait', 0.33509990490735364)
0.351766844606
(u'drink', 0.35176684460591617)
0.124759610985
(u'tweetdebate', 0.12475961098485762)
0.307174439314
(u'watch', 0.30717443931398541)
0.0
(u'roll', 0.0)
0.0
(u'okay', 0.0)
0.0
(u'one', 0.0)
0.0
(u'first', 0.0)
0.0
(u'say', 0.0)
0.0
(u'choice', 0.0)
0.0
(u'second', 0.0)
0.0
(u'actually', 0.0)
0.0
(u'slime', 0.0)
0.0
(u'yeah', 0.0)
0.0
(u'michdot', 0.0)
0.0
(u'ahg3', 0.0)
0.0
(u'attack', 0.0)
0.0
(u'heart', 0.0)
0.0
(u'prepare', 0.0)
0.124759610985
(u'tweetdebate', 0.12475961098485762)
0.0
(u'sorry', 0.0)
0.0
(u'moderator', 0.0)
0.0
(u'debate', 0.0)
0.124759610985
(u'tweetdebate', 0.12475961098485762)
0.0
(u'grrrrrrrrrrrrrrr', 0.0)
0.0
(u'wwwcurrentcomdebate', 0.0)
0.0
(u'screen', 0.0)
0.0
(u'black', 0.0)
0.0
(u'star', 0.0)
0.0
(u'current'

In [64]:
tweets[tweets["content"].str.contains("zoom")][['content']]

,content
tweet.id,
936687339,#debate08 - Did anyone else notice that the camera on Obama was always zoomed in more than the one on McCain? Purposeful?
